In [17]:
import pymysql
import csv
import pandas as pd
import sqlalchemy

In [28]:
file = '/users/eiofinova/Downloads/omim2phenotype_parsed4dataserver_withname.txt'

relations = pd.read_csv(file, sep='|')
relations

,OMIM disease ID,OMIM disease name,HPO,HPO name,evidence
0,OMIM:210100,"beta-aminoisobutyric acid, urinary excretion of",HP:0000007,Autosomal recessive inheritance,IEA
1,OMIM:210100,"beta-aminoisobutyric acid, urinary excretion of",HP:0003355,Aminoaciduria,IEA
2,OMIM:163600,nipples inverted,HP:0000006,Autosomal dominant inheritance,IEA
3,OMIM:163600,nipples inverted,HP:0003186,Inverted nipples,IEA
4,OMIM:615763,"#615763 cortical dysplasia, complex, with othe...",HP:0002365,Hypoplasia of the brainstem,TAS
...,...,...,...,...,...
95679,OMIM:607475,bothnia retinal dystrophy,HP:0000608,Macular degeneration,IEA
95680,OMIM:607475,bothnia retinal dystrophy,HP:0000662,Nyctalopia,TAS
95681,OMIM:607475,bothnia retinal dystrophy,HP:0000556,Retinal dystrophy,TAS
95682,OMIM:243400,"acetylation, slow",HP:0000007,Autosomal recessive inheritance,IEA


In [29]:
relations.evidence.unique()

array(['IEA', 'TAS', 'PCS'], dtype=object)

In [57]:
# Some of the OMIM IDs were combined with other IDs, so clean that up.
def find_omim(string):
    parts = string.split(';')
    if len(parts) <= 1:
        return string
    for part in parts:
        if part.startswith("OMIM"):
            return part
    return ""

find_omim('PMID:10080184;OMIM:185800')
relations['OMIM_id'] = relations['OMIM disease ID'].apply(find_omim)

In [58]:
relations

,OMIM disease ID,OMIM disease name,HPO,HPO name,evidence,OMIM_id
0,OMIM:210100,"beta-aminoisobutyric acid, urinary excretion of",HP:0000007,Autosomal recessive inheritance,IEA,OMIM:210100
1,OMIM:210100,"beta-aminoisobutyric acid, urinary excretion of",HP:0003355,Aminoaciduria,IEA,OMIM:210100
2,OMIM:163600,nipples inverted,HP:0000006,Autosomal dominant inheritance,IEA,OMIM:163600
3,OMIM:163600,nipples inverted,HP:0003186,Inverted nipples,IEA,OMIM:163600
4,OMIM:615763,"#615763 cortical dysplasia, complex, with othe...",HP:0002365,Hypoplasia of the brainstem,TAS,OMIM:615763
...,...,...,...,...,...,...
95679,OMIM:607475,bothnia retinal dystrophy,HP:0000608,Macular degeneration,IEA,OMIM:607475
95680,OMIM:607475,bothnia retinal dystrophy,HP:0000662,Nyctalopia,TAS,OMIM:607475
95681,OMIM:607475,bothnia retinal dystrophy,HP:0000556,Retinal dystrophy,TAS,OMIM:607475
95682,OMIM:243400,"acetylation, slow",HP:0000007,Autosomal recessive inheritance,IEA,OMIM:243400


In [76]:
db.close()

In [85]:
import db_config
dbconf = db_config.asimov_admin
db = pymysql.connect(dbconf["host"], dbconf["user"],
                         dbconf["password"], db='ppi')
cursor = db.cursor()

In [70]:
query = "drop table if exists tmp_ars"
cursor.execute(query)
query = "create table tmp_ars (OMIM_id varchar(50), OMIM_name varchar(1000), HPO_id varchar(50), comment varchar(5))"
cursor.execute(query)
query = """
INSERT INTO tmp_ars values %s
""" % ",".join(['("%s", "%s", "%s", "%s")' % (x[0], x[1], x[2], x[3]) for x in relations[['OMIM_id', 'OMIM disease name', 'HPO', 'evidence']].values])
cursor.execute(query)
db.commit()

In [67]:
query = "insert into attributes (namespace, external_id, name) select 'OMIM_DISEASE', OMIM_id, max(OMIM_name) from tmp_ars group by 2"
cursor.execute(query)
db.commit()

In [68]:
query = '''
insert into attribute_taxonomies (parent_id, child_id, distance, namespace)
select id, id, 0, namespace from attributes where namespace = 'OMIM_DISEASE'
'''
cursor.execute(query)
db.commit()

In [74]:
query = '''
insert into attribute_relations(attr1_id, attr2_id, comment)
SELECT omims.id, hpos.id, comment
FROM attributes omims
JOIN tmp_ars on omims.external_id = tmp_ars.OMIM_id
JOIN attributes hpos ON hpos.external_id = tmp_ars.HPO_id
'''
cursor.execute(query)
db.commit()

In [75]:
# Same thing, but with attr1 and attr2 switched, for symmetry.
query = '''
insert into attribute_relations(attr2_id, attr1_id, comment)
SELECT omims.id, hpos.id, comment
FROM attributes omims
JOIN tmp_ars on omims.external_id = tmp_ars.OMIM_id
JOIN attributes hpos ON hpos.external_id = tmp_ars.HPO_id
'''
cursor.execute(query)
db.commit()

In [84]:
file = '/users/eiofinova/Downloads/omim2gene_parsed4dataserver.txt'

omimgene = pd.read_csv(file, sep='|', dtype='object')
print(omimgene.columns)
omimgene["OMIM_id"] = "OMIM:" + omimgene["OMIM disease ID"]
omimgene

Index(['OMIM disease ID', 'disease name', 'gene symbol', 'entrezID'], dtype='object')


,OMIM disease ID,disease name,gene symbol,entrezID,OMIM_id
0,202110,17-alpha-hydroxylase/1720-lyase deficiency,CYP17A1,1586,OMIM:202110
1,300438,17-beta-hydroxysteroid dehydrogenase X deficiency,HSD17B10,3028,OMIM:300438
2,204750,2-aminoadipic 2-oxoadipic aciduria,DHTKD1,55526,OMIM:204750
3,610006,2-methylbutyrylglycinuria,ACADSB,36,OMIM:610006
4,273750,3-M syndrome 1,CUL7,9820,OMIM:273750
...,...,...,...,...,...
4247,612956,{Ventricular fibrillation paroxysmal familial 2},DPP6,1804,OMIM:612956
4248,612542,{Vitamin B12 plasma level QTL1},FUT2,2524,OMIM:612542
4249,606579,{Vitiligo-associated multiple autoimmune disea...,NLRP1,22861,OMIM:606579
4250,610379,{West nile virus susceptibility to},CCR5,1234,OMIM:610379


In [88]:
query = "drop table if exists tmp_og"
cursor.execute(query)
query = "create table tmp_og (OMIM_id varchar(50), entrez_id varchar(50))"
cursor.execute(query)
query = """
INSERT INTO tmp_og values %s
""" % ",".join(['("%s", "%s")' % (x[0], x[1]) for x in omimgene[['OMIM_id', 'entrezID']].values])
cursor.execute(query)
db.commit()

In [91]:
query = '''
INSERT INTO nodes_attributes (node_id, attribute_id)
SELECT DISTINCT nodes.id, attributes.id
FROM nodes JOIN tmp_og on nodes.external_id = tmp_og.entrez_id
JOIN attributes ON attributes.external_id = tmp_og.OMIM_id
'''
cursor.execute(query)
db.commit()

In [92]:
query = 'DROP TABLE tmp_ars'
cursor.execute(query)
query = 'DROP TABLE tmp_og'
cursor.execute(query)
db.commit()